In [46]:
from datasets import load_dataset
import random

In [18]:
DATASET= 'TheFinAI/FINQA_test'
DEFAULT_SUBSET = 10

In [48]:
 data_files = {
        "test": "https://raw.githubusercontent.com/czyssrs/FinQA/refs/heads/main/dataset/test.json"
 }

In [49]:
ds = load_dataset("json", data_files=data_files, split="test")

Generating test split: 0 examples [00:00, ? examples/s]

Failed to convert pandas DataFrame to Arrow Table from file '/Users/tavishisharma/.cache/huggingface/datasets/downloads/9b26cd02b7abeff97b335259f782a4c3c9324c3ee3bff07bff53bb5782f069a2' with error <class 'pyarrow.lib.ArrowInvalid'>: ("Could not convert 'yes' with type str: tried to convert to double", 'Conversion failed for column qa with type object')


DatasetGenerationError: An error occurred while generating the dataset

In [22]:
ds

Dataset({
    features: ['id', 'Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 1147
})

In [23]:
indices = list(range(len(ds)))
random.shuffle(indices)

In [24]:
n=DEFAULT_SUBSET

In [25]:
indices = indices[:n]

In [26]:
ds[indices[0]]

{'id': 'FINQA2',
 'Open-ended Verifiable Question': 'Please answer the given financial question based on the context.\nContext: undesignated hedges was $ 41.2 million and $ 42.1 million , respectively . the fair value of these hedging instruments in the company 2019s consolidated balance sheets as of october 29 , 2011 and october 30 , 2010 was immaterial . interest rate exposure management 2014 on june 30 , 2009 , the company entered into interest rate swap transactions related to its outstanding 5.0% ( 5.0 % ) senior unsecured notes where the company swapped the notional amount of its $ 375 million of fixed rate debt at 5.0% ( 5.0 % ) into floating interest rate debt through july 1 , 2014 . under the terms of the swaps , the company will ( i ) receive on the $ 375 million notional amount a 5.0% ( 5.0 % ) annual interest payment that is paid in two installments on the 1st of every january and july , commencing january 1 , 2010 through and ending on the maturity date ; and ( ii ) pay on

In [39]:
row = ds[indices[0]]

In [41]:
q = row["Open-ended Verifiable Question"]

In [42]:
q

'Please answer the given financial question based on the context.\nContext: undesignated hedges was $ 41.2 million and $ 42.1 million , respectively . the fair value of these hedging instruments in the company 2019s consolidated balance sheets as of october 29 , 2011 and october 30 , 2010 was immaterial . interest rate exposure management 2014 on june 30 , 2009 , the company entered into interest rate swap transactions related to its outstanding 5.0% ( 5.0 % ) senior unsecured notes where the company swapped the notional amount of its $ 375 million of fixed rate debt at 5.0% ( 5.0 % ) into floating interest rate debt through july 1 , 2014 . under the terms of the swaps , the company will ( i ) receive on the $ 375 million notional amount a 5.0% ( 5.0 % ) annual interest payment that is paid in two installments on the 1st of every january and july , commencing january 1 , 2010 through and ending on the maturity date ; and ( ii ) pay on the $ 375 million notional amount an annual three m

In [43]:
best = row['Ground-True Answer']

In [44]:
best

'0.09864'

In [21]:
prompts = []
refs = []
for i, idx in enumerate(indices):
    row = ds[int(idx)]
    q = row['data']["Question"]
    best = row['data']['Options'][row['data']["Correct Option"]]
    rid = f"truth-{i:04d}"
    prompts.append({
            "id": rid,
            "prompt": q,
            "system": "You are a concise, truthful assistant. Answer briefly and accurately."
        })
    refs.append({"id": rid, "references": best, "reference": best[0]})

In [22]:
idx

2964

In [23]:
ds[indices[0]]

{'problem': 'Trilochanapala, last king of the Kabul Shahi dynasty, ruled in the site of which modern country?',
 'answer': ['Afghanistan']}

In [24]:
ds[indices[2]]

{'problem': "who is a British physicist  and also a  British government's chief scientific adviser?",
 'answer': ['Frederick Alexander']}

In [25]:
refs

[{'id': 'truth-0000',
  'references': ['Afghanistan'],
  'reference': 'Afghanistan'},
 {'id': 'truth-0001', 'references': ['yes'], 'reference': 'yes'},
 {'id': 'truth-0002',
  'references': ['Frederick Alexander'],
  'reference': 'Frederick Alexander'},
 {'id': 'truth-0003',
  'references': ['"Orchard County"'],
  'reference': '"Orchard County"'},
 {'id': 'truth-0004',
  'references': ['political satire black comedy film'],
  'reference': 'political satire black comedy film'},
 {'id': 'truth-0005', 'references': ['six'], 'reference': 'six'},
 {'id': 'truth-0006',
  'references': ['Woody Woodpecker'],
  'reference': 'Woody Woodpecker'},
 {'id': 'truth-0007',
  'references': ['Fredric John Warburg'],
  'reference': 'Fredric John Warburg'},
 {'id': 'truth-0008',
  'references': ['Talia Shire'],
  'reference': 'Talia Shire'},
 {'id': 'truth-0009',
  'references': ['Old World fossil representatives'],
  'reference': 'Old World fossil representatives'}]

In [13]:
def prepare_truthful_qa(n: int = DEFAULT_SUBSET, 
                        split: str = "validation", 
                        seed: int = SEED) -> Tuple[str, str]:
    random.seed(seed)
    dataset= "truthful_qa"
    ds = load_dataset(dataset)[split]
    # Each row has 'question' and 'best_answer' (plus more fields); use best_answer as reference.
    indices = list(range(len(ds)))
    random.shuffle(indices)
    indices = indices[:n]
    prompts = []
    refs = []
    for i, idx in enumerate(indices):
        row = ds[int(idx)]
        q = row["question"]
        best = (row.get("best_answer") or "").strip()
        # TruthfulQA sometimes provides additional correct answers
        extra = row.get("correct_answers") or row.get("correct_answers_list") or []
        ref_list = []
        if best:
            ref_list.append(best)
        # ensure strings & strip
        for r in extra:
            if isinstance(r, str):
                r = r.strip()
                if r and r not in ref_list:
                    ref_list.append(r)
        rid = f"truth-{i:04d}"
        prompts.append({
            "id": rid,
            "prompt": q,
            "system": "You are a concise, truthful assistant. Answer briefly and accurately."
        })
        # Store both a list for robust scoring and a single field for back-compat
        refs.append({"id": rid, "references": ref_list, "reference": best})
    prompts_path = os.path.join(DATA_DIR, f"{dataset}_subset.jsonl")
    refs_path = os.path.join(DATA_DIR, f"{dataset}_refs.jsonl")
    write_jsonl(prompts_path, prompts)
    write_jsonl(refs_path, refs)

NameError: name 'SEED' is not defined